<a href="https://colab.research.google.com/github/ShafeeqAhamedS/PCA-Exp-5-Bubble-Sort-in-CUDA/blob/main/Bubble_and_Merge_sort.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-vzsr5ygt
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-vzsr5ygt
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 28f872a2f99a1b201bcd0db14fdbc5a496b9bfd7
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for nvcc4jupyter: filename=nvcc4jupyter-1.2.1-py3-none-any.whl size=10731 sha256=447e0940fae85115cdcc632697010fdf2bb3d88bd95975366a0e68f771b411d1
  Stored in directory: /tmp/pip-ephem-wheel-cache-6_zipq_s/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built nvcc4jupyter


In [2]:
%load_ext nvcc4jupyter

Detected platform "Colab". Running its setup...
Source files will be saved in "/tmp/tmpgmw429mm".


In [13]:
!nvidia-smi

Sun Nov 10 15:07:14 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P8              12W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [10]:
%%cuda
#include <stdio.h>
#include <stdlib.h>
#include <cuda.h>
#include <chrono>

// Kernel for Bubble Sort
__global__ void bubbleSortKernel(int *d_arr, int n) {
    int temp;
    int idx = threadIdx.x + blockIdx.x * blockDim.x;

    // Ensure we're using a single block for Bubble Sort
    if (blockIdx.x > 0) return;  // Only use the first block

    // Perform bubble sort across multiple passes
    for (int i = 0; i < n - 1; i++) {
        if (idx < n - 1 - i) {
            if (d_arr[idx] > d_arr[idx + 1]) {
                // Swap
                temp = d_arr[idx];
                d_arr[idx] = d_arr[idx + 1];
                d_arr[idx + 1] = temp;
            }
        }
        __syncthreads(); // Synchronize threads after each pass
    }
}
// Device function for merging arrays
__device__ void merge(int *arr, int left, int mid, int right, int *temp) {
    int i = left, j = mid + 1, k = left;

    while (i <= mid && j <= right) {
        if (arr[i] <= arr[j]) {
            temp[k++] = arr[i++];
        } else {
            temp[k++] = arr[j++];
        }
    }

    while (i <= mid) {
        temp[k++] = arr[i++];
    }

    while (j <= right) {
        temp[k++] = arr[j++];
    }

    for (i = left; i <= right; i++) {
        arr[i] = temp[i];
    }
}

// Kernel for Merge Sort
__global__ void mergeSortKernel(int *d_arr, int *d_temp, int n) {
    int tid = threadIdx.x + blockIdx.x * blockDim.x;

    // Loop over merging sizes: 1, 2, 4, 8, ...
    for (int size = 1; size < n; size *= 2) {
        int left = 2 * size * tid;
        if (left < n) {
            int mid = min(left + size - 1, n - 1);
            int right = min(left + 2 * size - 1, n - 1);

            merge(d_arr, left, mid, right, d_temp);  // Each thread merges one part
        }

        // Sync threads in the block to ensure all merges at this level are done
        __syncthreads();

        // Swap the array pointers
        if (tid == 0) {
            int *temp = d_arr;
            d_arr = d_temp;
            d_temp = temp;
        }

        // Sync again before starting next merge size
        __syncthreads();
    }
}

// Host function for merging arrays
void mergeHost(int *arr, int left, int mid, int right) {
    int i, j, k;
    int n1 = mid - left + 1;
    int n2 = right - mid;

    int *L = (int*)malloc(n1 * sizeof(int));
    int *R = (int*)malloc(n2 * sizeof(int));

    for (i = 0; i < n1; i++)
        L[i] = arr[left + i];
    for (j = 0; j < n2; j++)
        R[j] = arr[mid + 1 + j];

    i = 0;
    j = 0;
    k = left;

    while (i < n1 && j < n2) {
        if (L[i] <= R[j]) {
            arr[k] = L[i];
            i++;
        } else {
            arr[k] = R[j];
            j++;
        }
        k++;
    }

    while (i < n1) {
        arr[k] = L[i];
        i++;
        k++;
    }

    while (j < n2) {
        arr[k] = R[j];
        j++;
        k++;
    }

    free(L);
    free(R);
}

// Bubble Sort on GPU
void bubbleSort(int *arr, int n, int blockSize, int numBlocks) {
    int *d_arr;
    cudaMalloc((void**)&d_arr, n * sizeof(int));
    cudaMemcpy(d_arr, arr, n * sizeof(int), cudaMemcpyHostToDevice);

    // Start GPU timing
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);
    cudaEventRecord(start);

    bubbleSortKernel<<<numBlocks, blockSize>>>(d_arr, n);
    cudaDeviceSynchronize(); // Wait for GPU to finish

    cudaEventRecord(stop);
    cudaEventSynchronize(stop);

    float milliseconds = 0;
    cudaEventElapsedTime(&milliseconds, start, stop);

    cudaMemcpy(arr, d_arr, n * sizeof(int), cudaMemcpyDeviceToHost);
    cudaFree(d_arr);

    printf("Bubble Sort (GPU) took %f milliseconds\n", milliseconds);
}

// Merge Sort on GPU
void mergeSort(int *arr, int n, int blockSize, int numBlocks) {
    int *d_arr, *d_temp;
    cudaMalloc((void**)&d_arr, n * sizeof(int));
    cudaMalloc((void**)&d_temp, n * sizeof(int));
    cudaMemcpy(d_arr, arr, n * sizeof(int), cudaMemcpyHostToDevice);

    // Start GPU timing
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);
    cudaEventRecord(start);


    mergeSortKernel<<<numBlocks, blockSize>>>(d_arr, d_temp, n);
    cudaDeviceSynchronize(); // Wait for GPU to finish

    cudaEventRecord(stop);
    cudaEventSynchronize(stop);

    float milliseconds = 0;
    cudaEventElapsedTime(&milliseconds, start, stop);

    cudaMemcpy(arr, d_arr, n * sizeof(int), cudaMemcpyDeviceToHost);
    cudaFree(d_arr);
    cudaFree(d_temp);

    printf("Merge Sort (GPU) took %f milliseconds\n", milliseconds);
}

// Bubble Sort on CPU
void bubbleSortCPU(int *arr, int n) {
    auto start = std::chrono::high_resolution_clock::now();

    for (int i = 0; i < n - 1; i++) {
        for (int j = 0; j < n - i - 1; j++) {
            if (arr[j] > arr[j + 1]) {
                // Swap
                int temp = arr[j];
                arr[j] = arr[j + 1];
                arr[j + 1] = temp;
            }
        }
    }

    auto end = std::chrono::high_resolution_clock::now();
    std::chrono::duration<double, std::milli> duration = end - start;
    printf("Bubble Sort (CPU) took %f milliseconds\n", duration.count());
}

// Merge Sort on CPU
void mergeSortCPU(int *arr, int n) {
    auto start = std::chrono::high_resolution_clock::now();

    for (int size = 1; size < n; size *= 2) {
        int left = 0;
        while (left + size < n) {
            int mid = left + size - 1;
            int right = min(left + 2 * size - 1, n - 1);

            mergeHost(arr, left, mid, right); // Call host merge
            left += 2 * size;
        }
    }

    auto end = std::chrono::high_resolution_clock::now();
    std::chrono::duration<double, std::milli> duration = end - start;
    printf("Merge Sort (CPU) took %f milliseconds\n", duration.count());
}

// Main function
int main() {
    int n_array[] = {500, 1000};
    for (int i = 0; i < 2; i++) {
        int n = n_array[i];
        int *arr = (int*)malloc(n * sizeof(int));

        int blockSize_array[] = {16, 32};

        for (int i = 0; i < 2; i++) {

            int blockSize = blockSize_array[i]; // or higher, depending on the architecture
            int numBlocks = (n + blockSize - 1) / blockSize;

            printf("\nArray Size:%d\nBlock Size:%d\nNum Blocks:%d\n", n, blockSize, numBlocks);

            // Generating random array
            for (int i = 0; i < n; i++) {
                arr[i] = rand() % 1000;
            }

            // Bubble Sort CPU
            bubbleSortCPU(arr, n);

            // Generating random array again for GPU
            for (int i = 0; i < n; i++) {
                arr[i] = rand() % 1000;
            }

            // Bubble Sort GPU
            bubbleSort(arr, n, blockSize, numBlocks);

            // Generating random array again for GPU
            for (int i = 0; i < n; i++) {
                arr[i] = rand() % 1000;
            }

            // Merge Sort CPU
            mergeSortCPU(arr, n);

            // Generating random array again for Merge Sort
            for (int i = 0; i < n; i++) {
                arr[i] = rand() % 1000;
            }

            // Merge Sort GPU
            mergeSort(arr, n, blockSize, numBlocks);
            printf("\n");
        }
        free(arr);
    }
    return 0;
}


Array Size:500
Block Size:16
Num Blocks:32
Bubble Sort (CPU) took 0.796549 milliseconds
Bubble Sort (GPU) took 0.316128 milliseconds
Merge Sort (CPU) took 0.276864 milliseconds
Merge Sort (GPU) took 0.065606 milliseconds


Array Size:500
Block Size:32
Num Blocks:16
Bubble Sort (CPU) took 0.707811 milliseconds
Bubble Sort (GPU) took 0.112608 milliseconds
Merge Sort (CPU) took 0.261216 milliseconds
Merge Sort (GPU) took 0.064809 milliseconds


Array Size:1000
Block Size:16
Num Blocks:63
Bubble Sort (CPU) took 2.968457 milliseconds
Bubble Sort (GPU) took 0.208928 milliseconds
Merge Sort (CPU) took 0.504928 milliseconds
Merge Sort (GPU) took 0.139932 milliseconds


Array Size:1000
Block Size:32
Num Blocks:32
Bubble Sort (CPU) took 3.007277 milliseconds
Bubble Sort (GPU) took 0.208992 milliseconds
Merge Sort (CPU) took 0.500032 milliseconds
Merge Sort (GPU) took 0.141386 milliseconds


